<a href="https://colab.research.google.com/github/YasserJxxxx/RNN_Traductor/blob/main/Traductor_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import random

# --- 1. Vocabulario "desde cero" (SIN TILDES) ---

# --- FRASES BASICAS ---
frases_basicas = [
    ("hola", "hello"),
    ("como estas", "how are you"),
    ("buenos dias", "good morning"),
    ("buenas noches", "good night"),
    ("adios", "goodbye"),
    ("gracias", "thank you"),
    ("por favor", "please"),
    ("si", "yes"),
    ("no", "no"),
    ("como te llamas", "what is your name"),
    ("mi nombre es", "my name is"),
    ("donde estas", "where are you"),
    ("tengo hambre", "i am hungry"),
    ("esta lloviendo", "it is raining")
]

# --- ESTRUCTURAS GRAMATICALES (Sujeto-Verbo-Objeto) ---
sujetos = [
    ("el gato", "the cat"),
    ("el perro", "the dog"),
    ("un nino", "a boy"),
    ("una nina", "a girl"),
    ("el hombre", "the man"),
    ("la mujer", "the woman"),
    ("un pajaro", "a bird"),
    ("el estudiante", "the student")
]

verbos = [
    ("come", "eats"),
    ("ve", "sees"),
    ("juega con", "plays with"),
    ("quiere", "wants"),
    ("mira", "watches"),
    ("lee", "reads"),
    ("tiene", "has"),
    ("corre hacia", "runs to")
]

objetos = [
    ("la pelota", "the ball"),
    ("comida", "food"),
    ("un juguete", "a toy"),
    ("un arbol", "a tree"),
    ("el libro", "the book"),
    ("la casa", "the house"),
    ("agua", "water"),
    ("un auto", "a car")
]

# --- 2. Configuración ---
N_DATOS = 50000
ARCHIVO_FINAL_CSV = "dataset_sintetico_50k.csv"

# Probabilidad de que elijamos una frase basica (ej. 0.3 = 30% de las veces)
PROBABILIDAD_FRASE_BASICA = 0.3

print(f"Generando {N_DATOS} pares de oraciones sinteticas (sin tildes)...")

# --- 3. Proceso de generación y guardado ---

filas_generadas = []

for _ in range(N_DATOS):

    # Decidimos si generar una frase basica o una SVO
    if random.random() < PROBABILIDAD_FRASE_BASICA:
        # --- Generar Frase Basica ---
        par_frase = random.choice(frases_basicas)
        frase_es = par_frase[0]
        frase_en = par_frase[1]

    else:
        # --- Generar Frase SVO ---
        s = random.choice(sujetos)
        v = random.choice(verbos)
        o = random.choice(objetos)

        # Construimos la frase en espanol e ingles
        frase_es = f"{s[0]} {v[0]} {o[0]}."
        frase_en = f"{s[1]} {v[1]} {o[1]}."

    filas_generadas.append([frase_es, frase_en])

# Mezclamos la lista final para que no esten agrupadas
random.shuffle(filas_generadas)

# --- Guardamos todo en el CSV ---
try:
    with open(ARCHIVO_FINAL_CSV, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)

        # Escribimos el encabezado (header)
        writer.writerow(['espanol', 'ingles'])

        # Escribimos todas las filas generadas
        writer.writerows(filas_generadas)

    print(f"\n¡Éxito! 🥳")
    print(f"Dataset sintético guardado en: {ARCHIVO_FINAL_CSV}")

    # Imprimimos 5 ejemplos para verificar
    print("\n--- Vista previa de los datos generados ---")
    for i in range(5):
        print(filas_generadas[i])

except IOError as e:
    print(f"Error al escribir el archivo: {e}")
except Exception as e:
    print(f"Ocurrio un error inesperado: {e}")

Generando 50000 pares de oraciones sinteticas (sin tildes)...

¡Éxito! 🥳
Dataset sintético guardado en: dataset_sintetico_50k.csv

--- Vista previa de los datos generados ---
['el estudiante lee comida.', 'the student reads food.']
['el perro tiene la pelota.', 'the dog has the ball.']
['la mujer come la casa.', 'the woman eats the house.']
['buenos dias', 'good morning']
['un pajaro quiere el libro.', 'a bird wants the book.']


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import re
import pickle
import csv
from google.colab import drive

# --- 0. Montar Google Drive ---
drive.mount('/content/drive')

# --- 1. Configuración de Rutas ---

# RUTA DE ENTRADA (Dataset)
# Apunta al archivo LOCAL que creaste en la Celda 1
DATASET_FILE = "dataset_sintetico_50k.csv"

# RUTA DE SALIDA (Guardado)
# Apunta a tu carpeta de Google Drive
RUTA_BASE_DRIVE = "/content/drive/My Drive/TraductorRNN/"

# Crear la carpeta de Drive si no existe
if not os.path.exists(RUTA_BASE_DRIVE):
    os.makedirs(RUTA_BASE_DRIVE)
    print(f"Carpeta creada en: {RUTA_BASE_DRIVE}")

# Archivos que se guardarán/cargarán de Drive
MODEL_FILE = os.path.join(RUTA_BASE_DRIVE, 'traductor_rnn.keras')
TOKENIZER_SPA_FILE = os.path.join(RUTA_BASE_DRIVE, 'tokenizer_spa.pkl')
TOKENIZER_ENG_FILE = os.path.join(RUTA_BASE_DRIVE, 'tokenizer_eng.pkl')
MODEL_PARAMS_FILE = os.path.join(RUTA_BASE_DRIVE, 'model_params.pkl')
FEEDBACK_FILE = os.path.join(RUTA_BASE_DRIVE, 'feedback.csv')

# Parámetros del modelo
EMBEDDING_DIM = 256
LATENT_DIM = 256
BATCH_SIZE = 64
EPOCHS = 10

# --- 2. Funciones de Preprocesamiento ---

def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"([?.!,¿])", r" \1 ", texto)
    texto = re.sub(r'[" "]+', " ", texto)
    texto = texto.strip()
    return texto

def preparar_datos(dataset_path):
    """Carga y procesa el dataset desde la RUTA LOCAL."""
    print(f"Cargando y preparando datos desde {dataset_path}...")
    try:
        df = pd.read_csv(dataset_path)
    except FileNotFoundError:
        print(f"¡ERROR! No se encontró el dataset en: {dataset_path}")
        print("Por favor, asegúrate de ejecutar la 'Celda 1: Generar Dataset' primero.")
        return None, None, None, None

    espanol_textos = [limpiar_texto(str(txt)) for txt in df['espanol']]
    ingles_textos = [f"[start] {limpiar_texto(str(txt))} [end]" for txt in df['ingles']]

    tokenizer_spa = Tokenizer(filters='')
    tokenizer_spa.fit_on_texts(espanol_textos)

    tokenizer_eng = Tokenizer(filters='')
    tokenizer_eng.fit_on_texts(ingles_textos)

    # Guarda los tokenizers en DRIVE
    with open(TOKENIZER_SPA_FILE, 'wb') as f:
        pickle.dump(tokenizer_spa, f)
    with open(TOKENIZER_ENG_FILE, 'wb') as f:
        pickle.dump(tokenizer_eng, f)

    print(f"Tokenizers guardados en {RUTA_BASE_DRIVE}")
    return espanol_textos, ingles_textos, tokenizer_spa, tokenizer_eng

# --- 3. Función de Entrenamiento ---

def entrenar_y_guardar_modelo(espanol_textos, ingles_textos, tokenizer_spa, tokenizer_eng):
    """Construye, entrena y guarda el modelo en DRIVE."""

    print("Iniciando construcción y entrenamiento del modelo...")

    encoder_input_seq = tokenizer_spa.texts_to_sequences(espanol_textos)
    decoder_input_seq = tokenizer_eng.texts_to_sequences(ingles_textos)

    decoder_target_seq = [seq[1:] for seq in decoder_input_seq]

    max_len_spa = max(len(s) for s in encoder_input_seq)
    max_len_eng = max(len(s) for s in decoder_input_seq)

    # Guarda los parámetros del modelo en DRIVE
    with open(MODEL_PARAMS_FILE, 'wb') as f:
        pickle.dump({'max_len_spa': max_len_spa, 'max_len_eng': max_len_eng}, f)

    encoder_input_data = pad_sequences(encoder_input_seq, maxlen=max_len_spa, padding='post')
    decoder_input_data = pad_sequences(decoder_input_seq, maxlen=max_len_eng, padding='post')

    decoder_target_data_seq = pad_sequences(decoder_target_seq, maxlen=max_len_eng, padding='post')

    num_decoder_tokens = len(tokenizer_eng.word_index) + 1
    decoder_target_data = tf.keras.utils.to_categorical(
        decoder_target_data_seq, num_classes=num_decoder_tokens
    )

    print(f"Vocabulario Español: {len(tokenizer_spa.word_index) + 1} tokens")
    print(f"Vocabulario Inglés: {num_decoder_tokens} tokens")
    print(f"Max. secuencia Español: {max_len_spa}")
    print(f"Max. secuencia Inglés: {max_len_eng}")

    num_encoder_tokens = len(tokenizer_spa.word_index) + 1

    # --- ENCODER ---
    encoder_inputs = Input(shape=(None,), name='encoder_input')
    enc_embedding_layer = Embedding(num_encoder_tokens, EMBEDDING_DIM, name='encoder_embedding')
    enc_embedding_tensor = enc_embedding_layer(encoder_inputs)
    encoder_lstm_layer = LSTM(LATENT_DIM, return_state=True, name='encoder_lstm')
    _, state_h, state_c = encoder_lstm_layer(enc_embedding_tensor)
    encoder_states = [state_h, state_c]

    # --- DECODER ---
    decoder_inputs = Input(shape=(None,), name='decoder_input')
    dec_embedding_layer = Embedding(num_decoder_tokens, EMBEDDING_DIM, name='decoder_embedding')
    dec_embedding_tensor = dec_embedding_layer(decoder_inputs)
    decoder_lstm_layer = LSTM(LATENT_DIM, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, _, _ = decoder_lstm_layer(dec_embedding_tensor, initial_state=encoder_states)
    decoder_dense_layer = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
    decoder_outputs = decoder_dense_layer(decoder_outputs)

    # --- Modelo de Entrenamiento ---
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    print("\nEntrenando...")
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=0.2
    )

    # Guarda el modelo final en DRIVE
    model.save(MODEL_FILE)
    print(f"Modelo guardado en {MODEL_FILE}")
    return model

# --- 4. Funciones de Inferencia (Traducción) ---

def crear_modelos_de_inferencia(modelo_entrenado):
    """Crea los modelos separados de Encoder y Decoder."""

    print("Creando modelos de inferencia...")

    # --- ENCODER de Inferencia ---
    encoder_model_input = Input(shape=(None,), name='encoder_input_inf')
    enc_embedding_layer_inf = modelo_entrenado.get_layer('encoder_embedding')
    encoder_lstm_layer_inf = modelo_entrenado.get_layer('encoder_lstm')
    enc_emb_tensor_inf = enc_embedding_layer_inf(encoder_model_input)
    _, state_h_inf, state_c_inf = encoder_lstm_layer_inf(enc_emb_tensor_inf)
    encoder_model = Model(encoder_model_input, [state_h_inf, state_c_inf])

    # --- DECODER de Inferencia ---
    decoder_model_input = Input(shape=(1,), name='decoder_input_inf')
    decoder_state_h_input = Input(shape=(LATENT_DIM,), name='decoder_state_h_input')
    decoder_state_c_input = Input(shape=(LATENT_DIM,), name='decoder_state_c_input')
    decoder_states_inputs_inf = [decoder_state_h_input, decoder_state_c_input]

    dec_embedding_layer_inf = modelo_entrenado.get_layer('decoder_embedding')
    decoder_lstm_layer_inf = modelo_entrenado.get_layer('decoder_lstm')
    decoder_dense_layer_inf = modelo_entrenado.get_layer('decoder_dense')

    dec_emb_tensor_inf = dec_embedding_layer_inf(decoder_model_input)
    dec_outputs_inf, dec_state_h_inf, dec_state_c_inf = decoder_lstm_layer_inf(
        dec_emb_tensor_inf, initial_state=decoder_states_inputs_inf
    )
    dec_final_outputs = decoder_dense_layer_inf(dec_outputs_inf)

    decoder_model = Model(
        [decoder_model_input] + decoder_states_inputs_inf,
        [dec_final_outputs, dec_state_h_inf, dec_state_c_inf]
    )

    print("Modelos de Encoder y Decoder para inferencia creados.")
    return encoder_model, decoder_model

def traducir_frase(frase_es, encoder_model, decoder_model, tokenizer_spa, tokenizer_eng, max_len_spa, max_len_eng):
    """Toma una frase en español y la traduce."""

    frase_limpia = limpiar_texto(frase_es)
    input_seq_raw = tokenizer_spa.texts_to_sequences([frase_limpia])
    input_seq = pad_sequences(input_seq_raw, maxlen=max_len_spa, padding='post')

    if not np.any(input_seq): # Si todas las palabras son desconocidas
        return "[Error: No reconozco ninguna de esas palabras]"

    states_value = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer_eng.word_index['[start]']

    stop_condition = False
    decoded_sentence = ""
    idx_a_palabra_eng = {v: k for k, v in tokenizer_eng.word_index.items()}

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = idx_a_palabra_eng.get(sampled_token_index, '[UNK]')

        if (sampled_word == '[end]' or
           len(decoded_sentence.split()) > max_len_eng):
            stop_condition = True
        else:
            decoded_sentence += " " + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

# --- 5. Bucle Principal de Ejecución ---

def main():

    # Comprueba si el MODELO existe en DRIVE
    if os.path.exists(MODEL_FILE) and os.path.exists(TOKENIZER_SPA_FILE):
        print("Cargando modelo y tokenizers existentes desde Drive...")
        modelo_entrenado = load_model(MODEL_FILE)

        with open(TOKENIZER_SPA_FILE, 'rb') as f:
            tokenizer_spa = pickle.load(f)
        with open(TOKENIZER_ENG_FILE, 'rb') as f:
            tokenizer_eng = pickle.load(f)
        with open(MODEL_PARAMS_FILE, 'rb') as f:
            params = pickle.load(f)
            max_len_spa = params['max_len_spa']
            max_len_eng = params['max_len_eng']

    else:
        print("No se encontró un modelo en Drive. Entrenando uno nuevo...")
        # Prepara los datos desde el CSV LOCAL
        datos = preparar_datos(DATASET_FILE)
        if datos[0] is None:
            return # Detener si el CSV no se encontró

        spa_text, eng_text, tokenizer_spa, tokenizer_eng = datos

        # Entrena y GUARDA EN DRIVE
        modelo_entrenado = entrenar_y_guardar_modelo(
            spa_text, eng_text, tokenizer_spa, tokenizer_eng
        )
        # Carga los params que acabamos de guardar
        with open(MODEL_PARAMS_FILE, 'rb') as f:
            params = pickle.load(f)
            max_len_spa = params['max_len_spa']
            max_len_eng = params['max_len_eng']

    # Crear modelos de Inferencia
    encoder_model_inf, decoder_model_inf = crear_modelos_de_inferencia(
        modelo_entrenado,
    )
    print("¡Traductor listo!")

    # Bucle de Interacción
    print("\n--- Traductor Interactivo (RNN) ---")
    print("Escribe 'salir' para terminar.")

    while True:
        frase_original = input("\nEscribe en español: ")
        if frase_original.lower() == 'salir':
            break
        if not frase_original.strip():
            continue

        traduccion = traducir_frase(
            frase_original,
            encoder_model_inf,
            decoder_model_inf,
            tokenizer_spa,
            tokenizer_eng,
            max_len_spa,
            max_len_eng
        )
        print(f"Traducción: {traduccion}")

        # Recolección de Feedback (se guarda en DRIVE)
        feedback = input("¿Traducción correcta? (s/n): ").lower()

        if feedback == 'n':
            correccion = input(f"Escribe la traducción correcta para '{frase_original}': ")
            nueva_fila = [frase_original, correccion]
            file_exists = os.path.exists(FEEDBACK_FILE)

            with open(FEEDBACK_FILE, 'a', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                if not file_exists:
                    writer.writerow(['espanol', 'ingles'])
                writer.writerow(nueva_fila)
            print(f"¡Gracias! Corrección guardada en '{FEEDBACK_FILE}'.")
        elif feedback == 's':
            print("¡Genial! 👍")

# --- ¡Ejecutar todo! ---
if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cargando modelo y tokenizers existentes desde Drive...
Creando modelos de inferencia...
Modelos de Encoder y Decoder para inferencia creados.
¡Traductor listo!

--- Traductor Interactivo (RNN) ---
Escribe 'salir' para terminar.

Escribe en español: español
Traducción: [Error: No reconozco ninguna de esas palabras]
¿Traducción correcta? (s/n): n
Escribe la traducción correcta para 'español': ingles
¡Gracias! Corrección guardada en '/content/drive/My Drive/TraductorRNN/feedback.csv'.

Escribe en español: hola


Traducción: hello
¿Traducción correcta? (s/n): s
¡Genial! 👍

Escribe en español: como estas 
Traducción: how are you
¿Traducción correcta? (s/n): s
¡Genial! 👍

Escribe en español: n
Traducción: [Error: No reconozco ninguna de esas palabras]
¿Traducción correcta? (s/n): n
Escribe la traducción correcta para 'n': n
¡Gracias! Corrección guardada en '/content/drive/My Drive/TraductorRNN/feedback.csv'.

Escribe en español: no
Traducción: no
¿Traducción correcta? (s/n): s
¡Genial! 👍

Escribe en español: listo
Traducción: [Error: No reconozco ninguna de esas palabras]
¿Traducción correcta? (s/n): s
¡Genial! 👍

Escribe en español: ready
Traducción: [Error: No reconozco ninguna de esas palabras]
¿Traducción correcta? (s/n): mi nombre es

Escribe en español: mi nombre es
Traducción: my name is
¿Traducción correcta? (s/n): s
¡Genial! 👍

Escribe en español: tengo hambre
Traducción: i am hungry
